# A.4a - Synthetic Depth Only Training

**Experiment**: A.4a
**Input**: Synthetic depth maps only (3-channel, generated by Depth-Anything-V2)
**Objective**: Test if synthetic depth can replace real depth sensor
**Comparison**: Compare with A.2 (Real Depth Only): mAP50=0.628, mAP50-95=0.226

**Prerequisites**:
1. Run `generate_synthetic_depth.ipynb` first
2. Run `prepare_synthetic_depth_data.py` to process depth maps

---

In [ ]:
# Setup
import os
import torch
import numpy as np
from pathlib import Path
from ultralytics import YOLO
import pandas as pd
from datetime import datetime

IS_KAGGLE = os.path.exists('/kaggle/input')
BASE_PATH = Path('/kaggle/working' if IS_KAGGLE else r'd:\Work\Assisten Dosen\Anylabel\Experiments')
DATASET_PATH = Path('/kaggle/input/ffb-synthetic-depth' if IS_KAGGLE else BASE_PATH / 'datasets' / 'ffb_localization_depth_synthetic')

print(f"Environment: {'Kaggle' if IS_KAGGLE else 'Local'}")
print(f"Dataset: {DATASET_PATH}")
print(f"CUDA: {torch.cuda.is_available()}")

In [ ]:
# Create YAML config
config_path = BASE_PATH / 'configs' / 'ffb_localization_depth_synthetic.yaml'
config_path.parent.mkdir(exist_ok=True)

yaml_content = f"""path: {DATASET_PATH.as_posix()}
train: images/train
val: images/val
test: images/test
nc: 1
names: ['fresh_fruit_bunch']
"""
config_path.write_text(yaml_content)
print(f"Config saved: {config_path}")

In [ ]:
# Training config
CONFIG = {
    'model': 'yolo11n.pt',
    'epochs': 50,
    'batch': 16,
    'imgsz': 640,
    'device': 0 if torch.cuda.is_available() else 'cpu',
}
SEEDS = [42, 123]
EXP_PREFIX = 'exp_a4a_synthetic'

## Training Runs (Seeds 42 & 123)

In [ ]:
# Train both seeds
for seed in SEEDS:
    print(f"\n{'='*60}\nTRAINING SEED {seed}\n{'='*60}\n")
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    model = YOLO(CONFIG['model'])
    results = model.train(
        data=str(config_path),
        epochs=CONFIG['epochs'],
        batch=CONFIG['batch'],
        imgsz=CONFIG['imgsz'],
        device=CONFIG['device'],
        name=f"{EXP_PREFIX}_seed{seed}",
        seed=seed,
        exist_ok=True
    )
    print(f"✅ Seed {seed} complete")

## Evaluation on Test Set

In [ ]:
# Evaluate
results_dict = {}
for seed in SEEDS:
    model_path = BASE_PATH / 'runs' / 'detect' / f"{EXP_PREFIX}_seed{seed}" / 'weights' / 'best.pt'
    model = YOLO(str(model_path))
    metrics = model.val(data=str(config_path), split='test', batch=CONFIG['batch'])
    
    results_dict[seed] = {
        'mAP50': metrics.box.map50,
        'mAP50-95': metrics.box.map,
        'precision': metrics.box.mp,
        'recall': metrics.box.mr
    }

# Summary
df = pd.DataFrame(results_dict).T
avg = df.mean()
avg.name = 'Average'
df = pd.concat([df, avg.to_frame().T])

print("\n" + "="*60)
print("A.4a SYNTHETIC DEPTH ONLY - RESULTS")
print("="*60 + "\n")
print(df.to_string(float_format=lambda x: f"{x:.3f}"))

# Comparison
print("\n" + "="*60)
print("COMPARISON: REAL DEPTH vs SYNTHETIC DEPTH")
print("="*60 + "\n")
comparison = pd.DataFrame({
    'A.2 Real Depth': {'mAP50': 0.628, 'mAP50-95': 0.226},
    'A.4a Synthetic': {'mAP50': avg['mAP50'], 'mAP50-95': avg['mAP50-95']}
}).T
print(comparison.to_string(float_format=lambda x: f"{x:.3f}"))

# Save
output_file = BASE_PATH / 'kaggleoutput' / 'test_synthetic_depth.txt'
output_file.parent.mkdir(exist_ok=True, parents=True)
with open(output_file, 'w') as f:
    f.write(f"A.4a Synthetic Depth Only Results\nGenerated: {datetime.now()}\n\n")
    f.write(df.to_string(float_format=lambda x: f"{x:.3f}"))
    f.write("\n\nComparison:\n")
    f.write(comparison.to_string(float_format=lambda x: f"{x:.3f}"))
print(f"\n✅ Saved: {output_file}")